# Introduction

This notebook aims to evaluate the performance of various classification algorithms, fine-tune their hyperparameters, and select the best-performing model. Given the imbalanced nature of the dataset, class weighting (`class_weight='balanced'`) will be applied to enhance model performance.

The algorithms to be evaluated include:

- Logistic Regression
- Decision Tree
- Random Forest
- XGBoost

Each model will undergo:
1. Baseline training with default hyperparameters.
2. Hyperparameter tuning to optimize performance.
3. Validation using metrics such as AUC to identify overfitting and select the final model.

## Important Note:
**(Strongly correlated features)**

During initial tests, we observed the model achieved high performance due to features strongly correlated with the target variable, such as economic participation and hours worked, reflecting real-world factors influencing school dropout rates. To better evaluate model performance and for illustrative purposes in this exercise, we will exclude the most predictive features, allowing us to practice model selection, training, and tuning without their dominant influence. The following cells will demonstrate this condition and make the aforementioned adjustments.

Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.exceptions import ConvergenceWarning

# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### Base Model

#### Logistic Regression with default parameters

Load data

In [2]:
# Load data
df = pd.read_csv('G:\Mi unidad\###_ ML Zoomcamp 2024\enape_post_eda.csv')

Split dataset (60/20/20)

In [3]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.dropout.values
y_val = df_val.dropout.values
y_test = df_test.dropout.values

del df_train['dropout']
del df_val['dropout']
del df_test['dropout']

In [4]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(11983, 41)
(3995, 41)
(3995, 41)


Train model

In [5]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

Check accuracy

In [6]:
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict_proba(X_val)[:, 1]
dropout_prediction = (y_pred >= 0.5)
(y_val == dropout_prediction).mean()

np.float64(0.9987484355444305)

In [7]:
report = classification_report(y_val, dropout_prediction)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3907
           1       1.00      0.94      0.97        88

    accuracy                           1.00      3995
   macro avg       1.00      0.97      0.99      3995
weighted avg       1.00      1.00      1.00      3995



In [8]:
roc_auc_score(y_val,dropout_prediction).round(3)

np.float64(0.972)

It seems that our model is overfitting, we'll check with the test split and see how it handles new data.


In [9]:
test_dict = df_test.to_dict(orient='records')
X_test = dv.transform(test_dict)

y_pred = model.predict_proba(X_test)[:, 1]
dropout_prediction = (y_pred >= 0.5)
(y_test == dropout_prediction).mean()

np.float64(0.9987484355444305)

In [10]:
report = classification_report(y_test, dropout_prediction)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3931
           1       0.98      0.94      0.96        64

    accuracy                           1.00      3995
   macro avg       0.99      0.97      0.98      3995
weighted avg       1.00      1.00      1.00      3995



In [11]:
roc_auc_score(y_test,dropout_prediction).round(3)

np.float64(0.969)

The model performs well on new data, indicating that it is generalizing effectively. Therefore, we can proceed with a cross-validation exercise.

Additionally, we know (as stated in the [Corelation Analysis with the Target Variable](https://github.com/Maxkaizo/---_-ML-Zoomcamp-2024/blob/main/2_eda.ipynb)) that certain features, such as economic participation, economic consequences, and work hours, have a strong influence on dropout rates. To further illustrate the impact of feature selection, we will remove some of these features in the next steps.

#### Cross Validation

In [12]:
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = np.std(scores)

print(scores)
print("Cross-Validation Accuracy (mean):", mean_accuracy)
print("Cross-Validation Accuracy (std):", std_accuracy)


[0.99833125 0.99666249 0.99791406 0.99791319 0.99958264]
Cross-Validation Accuracy (mean): 0.9980807255591471
Cross-Validation Accuracy (std): 0.0009365603198218517


#### Base Model Conclusions

The model performs exceptionally well, mostly due to the presence of highly correlated features. After testing it with cross-validation and new data, we can conclude that it generalizes effectively. This aligns with the real-world scenario, where it is intuitive to expect that students with excessive work hours, high economic participation, and significant economic consequences are more likely to drop out.

#### Feature exclusion

As stated before, we'll exclude some features in order to run the model's selection and fine tunning process

In [13]:
# Load data
df = pd.read_csv('G:\Mi unidad\###_ ML Zoomcamp 2024\enape_post_eda.csv')

In [14]:
columns_to_drop = [
    'em_hw_projects',
    'em_tests',
    #'economic_participation',
    #'economic_consequences'
    ] 
df_full_train = df_full_train.drop(columns=columns_to_drop, axis=1)
df_train = df_train.drop(columns=columns_to_drop, axis=1)
df_val = df_val.drop(columns=columns_to_drop, axis=1)
df_test = df_test.drop(columns=columns_to_drop, axis=1)

In [15]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(11983, 39)
(3995, 39)
(3995, 39)


Now we have a poor performance model and we can go on and try to improve it.

# Model Selection

## Logistic Regression

### Define Functions (Train, Predict, Evaluation Metrics)

Train function

In [16]:
def train(df_train, y_train, C=1.0,cw=None):
    dict = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dict)

    model = LogisticRegression(C=C, max_iter=1000,class_weight=cw)
    model.fit(X_train, y_train)
    
    return dv, model

Predict Function

In [17]:
def predict(df, dv, model):
    dict = df.to_dict(orient='records')

    X = dv.transform(dict)
    y_pred = model.predict_proba(X)[:, 1]
    dropout_prediction = (y_pred >= 0.5)

    return y_pred, dropout_prediction

In [18]:
def eval_metrics(y, dropout_prediction):

    gral_accuracy = (y == dropout_prediction).mean()
    report = classification_report(y, dropout_prediction, output_dict=True)
    auc = roc_auc_score(y, dropout_prediction).round(3)
    
    # Filtrar solo la clase 1 del reporte
    class_1_report = report["1"]
    
    metrics_dict = {
        "gral_accuracy": gral_accuracy,
        "class_1_report": class_1_report,
        "auc": auc
    }
    
    return metrics_dict

### Base (Poor) Performance

In [19]:
dv, model = train(df_train, y_train, C=1.0,cw=None)
y_pred, dropout_prediction = predict(df_val, dv, model)
eval_metrics(y_val, dropout_prediction)

{'gral_accuracy': np.float64(0.9799749687108886),
 'class_1_report': {'precision': 0.6666666666666666,
  'recall': 0.18181818181818182,
  'f1-score': 0.2857142857142857,
  'support': 88.0},
 'auc': np.float64(0.59)}

As stated before, this model shows a poor performance. It has an apparently good accuracy score, but our main metric is recall and its to low, also our auc shows barely a better performance than a random model.

### Class Weight Balancing

Our first improvement is to apply balanced weights, this technique apply deals with imbalance by applying a higher ponderation to the minority class, in this case the dropout class

In [20]:
dv, model = train(df_train, y_train, C=1.0,cw="balanced")
y_pred, dropout_prediction = predict(df_val, dv, model)
eval_metrics(y_val, dropout_prediction)

{'gral_accuracy': np.float64(0.8578222778473091),
 'class_1_report': {'precision': 0.1261682242990654,
  'recall': 0.9204545454545454,
  'f1-score': 0.2219178082191781,
  'support': 88.0},
 'auc': np.float64(0.888)}

### Hyperparameter Selection

Now the model shows way better recall and auc, but now it shows poor precision, so now we'll try to find a better balance by:
- Using balanced weigths
- Adjust the decision threshold
- Trying some regularization values

In [21]:
c_values = [1.0, 0.01, 0.001, 0.0001, 0.00001]
thresholds = [0.3, 0.5, 0.7, 0.8, 0.9]

eval_metrics_dict = []

for c in c_values:
    for t in thresholds:
        dv, model = train(df_train, y_train, C=c, cw="balanced")
        y_pred, dropout_prediction = predict(df_val, dv, model)
        
        # Evalúa las métricas
        eval_metrics_result = eval_metrics(y_val, dropout_prediction)
        
        # Almacena los resultados en el diccionario
        eval_metrics_dict.append({
            "C": c,
            "Threshold": t,
            "result": eval_metrics_result
        })

In [23]:
rows = []
for item in eval_metrics_dict:
    c_value = item['C']
    threshold = item['Threshold']
    gral_accuracy = item['result']['gral_accuracy']
    auc = item['result']['auc']
    class_1_report = item['result']['class_1_report']
    rows.append({
        "C": c_value,
        "Threshold": threshold,
        "gral_accuracy": gral_accuracy,
        "auc": auc,
        "precision": class_1_report['precision'],
        "recall": class_1_report['recall'],
        "f1_score": class_1_report['f1-score'],
        "support": class_1_report['support']
    })

# Crear el DataFrame
df_results = pd.DataFrame(rows)

In [32]:
df_results.head(10).sort_values(by='recall', ascending=False)

,C,Threshold,gral_accuracy,auc,precision,recall,f1_score,support
0,1.00,0.3,0.857822,0.888,0.126168,0.920455,0.221918,88.0
1,1.00,0.5,0.857822,0.888,0.126168,0.920455,0.221918,88.0
2,1.00,0.7,0.857822,0.888,0.126168,0.920455,0.221918,88.0
3,1.00,0.8,0.857822,0.888,0.126168,0.920455,0.221918,88.0
4,1.00,0.9,0.857822,0.888,0.126168,0.920455,0.221918,88.0
5,0.01,0.3,0.848561,0.839,0.110106,0.829545,0.194407,88.0
6,0.01,0.5,0.848561,0.839,0.110106,0.829545,0.194407,88.0
7,0.01,0.7,0.848561,0.839,0.110106,0.829545,0.194407,88.0
8,0.01,0.8,0.848561,0.839,0.110106,0.829545,0.194407,88.0
9,0.01,0.9,0.848561,0.839,0.110106,0.829545,0.194407,88.0


We'll select the top 2 combinations as candidates

### SMOTE Testing

Now we'll try another technique for dealing with our imbalanced dataset, this one is called SMOTE (Synthetic Minority Oversampling Technique), which is a method that generates synthetic samples for the minority class. It works by interpolating new samples between existing minority class instances, rather than simply duplicating them, which helps create a more balanced dataset and improves model performance without overfitting.

In [25]:
from imblearn.over_sampling import SMOTE

In [26]:
def train_wsmote(df_train, y_train, C=1.0,cw=None):
    dict = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dict)
    
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model = LogisticRegression(C=C, max_iter=1000,class_weight=cw)
    model.fit(X_train, y_train)
    
    return dv, model

In [27]:
c_values = [1.0, 0.01, 0.001, 0.0001, 0.00001]
thresholds = [0.3, 0.5, 0.7, 0.8, 0.9]

eval_metrics_dict = []

for c in c_values:
    for t in thresholds:
        dv, model = train_wsmote(df_train, y_train, C=c, cw=None) # As we're using SMOTE, in this case we won't use class weights
        y_pred, dropout_prediction = predict(df_val, dv, model)
        
        # Evalúa las métricas
        eval_metrics_result = eval_metrics(y_val, dropout_prediction)
        
        # Almacena los resultados en el diccionario
        eval_metrics_dict.append({
            "C": c,
            "Threshold": t,
            "result": eval_metrics_result
        })

In [28]:
rows = []
for item in eval_metrics_dict:
    c_value = item['C']
    threshold = item['Threshold']
    gral_accuracy = item['result']['gral_accuracy']
    auc = item['result']['auc']
    class_1_report = item['result']['class_1_report']
    rows.append({
        "C": c_value,
        "Threshold": threshold,
        "gral_accuracy": gral_accuracy,
        "auc": auc,
        "precision": class_1_report['precision'],
        "recall": class_1_report['recall'],
        "f1_score": class_1_report['f1-score'],
        "support": class_1_report['support']
    })

# Crear el DataFrame
df_results_wsmote = pd.DataFrame(rows)

In [29]:
df_results.head(5).sort_values(by='precision', ascending=False)

,C,Threshold,gral_accuracy,auc,precision,recall,f1_score,support
0,1.0,0.3,0.857822,0.888,0.126168,0.920455,0.221918,88.0
1,1.0,0.5,0.857822,0.888,0.126168,0.920455,0.221918,88.0
2,1.0,0.7,0.857822,0.888,0.126168,0.920455,0.221918,88.0
3,1.0,0.8,0.857822,0.888,0.126168,0.920455,0.221918,88.0
4,1.0,0.9,0.857822,0.888,0.126168,0.920455,0.221918,88.0


In [30]:
df_results_wsmote.head(5).sort_values(by='precision', ascending=False)

,C,Threshold,gral_accuracy,auc,precision,recall,f1_score,support
0,1.0,0.3,0.872591,0.852,0.128748,0.829545,0.222901,88.0
1,1.0,0.5,0.872591,0.852,0.128748,0.829545,0.222901,88.0
2,1.0,0.7,0.872591,0.852,0.128748,0.829545,0.222901,88.0
3,1.0,0.8,0.872591,0.852,0.128748,0.829545,0.222901,88.0
4,1.0,0.9,0.872591,0.852,0.128748,0.829545,0.222901,88.0


This indicates that while SMOTE helps the model to be more precise in its predictions for the minority class (fewer false positives), it comes at the cost of a significant drop in recall (more false negatives). Since recall is often critical in imbalanced classification problems, especially when identifying minority class instances is a priority, the trade-off makes SMOTE less suitable in this case. So we won't use it further